# Imports

In [1]:
import pandas as pd
import numpy as np
import os

# Merge Data

In [271]:
def merge_csv(base_path, subdir, out_path, file_type):
    dir_path = os.path.join(base_path, subdir)
    all_files = sorted(os.listdir(dir_path))

    merged_df = pd.DataFrame()

    for file in all_files:
        file_path = os.path.join(dir_path, file)
        #print(file) # debugging
        if file_type == 'te': # for train equipment
            narr_columns = [f'NARR{i}' for i in range(1, 16)]
            cols = ['YEAR', 'MONTH', 'DAY', 'TYPE', 'CARSDMG', 
                    'CARSHZD', 'STATE', 'TEMP', 'VISIBLTY', 'WEATHER',
                    'TRNSPD', 'TONS', 'TYPEQ', 'TYPTRK', 'CAUSE', 
                    'CASKLDRR', 'CASINJRR', 'ACCDMG', 'Latitude', 
                    'Longitud'] + narr_columns
            df = pd.read_csv(file_path, index_col=False, dtype=str) # csv
            df = df[cols]
            merged_df = pd.concat([merged_df, df], axis=0)
            #print(merged_df['YEAR'].unique()) # debugging
        elif file_type == 'app': # for appendix
                df = pd.read_excel(file_path, sheet_name='Sheet1', dtype=str) #xls
                df.columns = df.columns.str.upper()
                cols = ['CODE', 'DESCRIPTION', 'CATEGORY', 'TITLE']
                df = df[cols]
                merged_df = pd.concat([merged_df, df], axis=0)
        else:
            print(f"Invalid file type: {file_type}")
            
    output_file = os.path.join(out_path, f'{subdir}_merged.csv')
    merged_df.to_csv(output_file, index=False)


path = './data/'
subdirectories = ['TrainEquipment', 'TrainEquipmentAppendix']

for subdir in subdirectories:
    file_type = "te" if subdir=='TrainEquipment' else "app"
    #print(file_type) # debugging
    merge_csv(path, subdir, path, file_type)

# Clean Data

## Train Equipment

In [17]:
# ref: https://railroads.dot.gov/forms-guides-publications/guides/618054-rail-equipment-accidentincident-thru-52011-206kb
# https://www.bls.gov/respondents/mwr/electronic-data-interchange/appendix-d-usps-state-abbreviations-and-fips-codes.htm
def convert_to_date(year):
    year = int(year)
    if 75 <= year <= 99:
        return 1900 + year
    elif 0 <= year <= 22:
        return 2000 + year
    return None

def convert_type(type):
    types = {
        1: "Derailment",
        2: "Head on Collision",
        3: "Rearend Collision",
        4: "Side Collision",
        5: "Raking Collision",
        6: "Broken Train Collision",
        7: "Hwy-Rail Crossing",
        8: "RR Grade Crossing",
        9: "Obstruction",
        10: "Explosive Detonation",
        11: "Fire / Violent Rupture",
        12: "Other Impacts",
        13: "Other"
    }
    type = int(type)
    if type in types:
        return types[type]
    else:
        return None
    
def convert_state(fips_code):
    fips_to_state = {
        '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT',
        '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL',
        '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
        '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE',
        '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
        '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD',
        '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV',
        '55': 'WI', '56': 'WY'
    }
    
    if fips_code in fips_to_state:
        return fips_to_state[fips_code]
    else:
        return None
    
def convert_vis(vis):
    vis_types = {
        '1': "Dawn",
        '2': "Day",
        '3': "Dusk",
        '4': "Dark"
    }
    if pd.isnull(vis):
        return np.nan
    elif str(vis) in vis_types:
        return vis_types[str(vis)]
    else:
        return np.nan

def convert_weather(weather):
    weather_types = {
        '1': "Clear",
        '2': "Cloudy",
        '3': "Rain",
        '4': "Fog",
        '5': "Sleet",
        '6': "Snow"
    }
    if pd.isnull(weather):
        return np.nan
    elif str(weather) in weather_types:
        return weather_types[str(weather)]
    else:
        return np.nan
    
def convert_eq(eq):
    eq_types = {
        "1": "Freight Train",
        "2": "Passenger Train",
        "3": "Commuter Train",
        "4": "Work Train",
        "5": "Single Car",
        "6": "Cut of Cars",
        "7": "Yard / Switching", # at a yard or while switching
        "8": "Light Loco",
        "9": "Maint / Inspect Car", 
        "A": "Spec. MoW Eq." # Maintenance-of-Way 
    }
    if eq in eq_types:
        return eq_types[eq]
    else:
        return None
    
def convert_track(track):
    track_types = {
        '1': "Main",
        '2': "Yard",
        '3': "Siding",
        '4': "Industry"
    }
    if pd.isnull(track):
        return np.nan
    elif str(track) in track_types:
        return track_types[str(track)]
    else:
        return np.nan

In [18]:
te = pd.read_csv('./data/TrainEquipment_merged.csv', dtype=str)
te.dropna(subset=['VISIBLTY', 'WEATHER', 'TYPTRK', 'TRNSPD'], inplace=True) # drop NaNs

## merge YEAR, MONTH, DAY cols to a DATE col
te = te[te['DAY'] != '00'] # filter out invalid rows where day == 00
te['YEAR'] = te['YEAR'].apply(convert_to_date)
te['DATE'] = pd.to_datetime(te[['YEAR', 'MONTH', 'DAY']], format='%Y-%m-%d')
te.drop(columns=['YEAR', 'MONTH', 'DAY'], inplace=True) # drop YEAR, MONTH, DAY cols
te = te.sort_values(by='DATE')

## convert encoding to informative values based on documentation
te['TYPE'] = te['TYPE'].apply(convert_type)
te['STATE'] = te['STATE'].apply(convert_state)
te['VISIBLTY'] = te['VISIBLTY'].apply(convert_vis)
te['WEATHER'] = te['WEATHER'].apply(convert_weather)
te['TYPEQ'] = te['TYPEQ'].apply(convert_eq)
te['TYPTRK'] = te['TYPTRK'].apply(convert_track)

## convert numeric cols to int
te['CARSDMG'] = pd.to_numeric(te['CARSDMG'], errors='coerce').astype(int)
te['CARSHZD'] = pd.to_numeric(te['CARSHZD'], errors='coerce').astype(int)
te['TEMP'] = pd.to_numeric(te['TEMP'], errors='coerce').astype(int)
te['TRNSPD'] = pd.to_numeric(te['TRNSPD'], errors='coerce').astype(int)
te['TONS'] = pd.to_numeric(te['TONS'], errors='coerce').astype(int)
te['CASKLDRR'] = pd.to_numeric(te['CASKLDRR'], errors='coerce').astype(int)
te['CASINJRR'] = pd.to_numeric(te['CASINJRR'], errors='coerce').astype(int)
te['ACCDMG'] = pd.to_numeric(te['ACCDMG'], errors='coerce').astype(int)

te.dropna(subset=['VISIBLTY', 'WEATHER', 'TYPTRK', 'TRNSPD'], inplace=True) # drop NaNs resulting from conversions

In [64]:
te_app = pd.read_csv('./data/TrainEquipmentAppendix_merged.csv', dtype=str)

te_app['CODE'] = te_app['CODE'].str.strip()

## merge based on accident code
merged_df = pd.merge(te, te_app, left_on='CAUSE', right_on='CODE', how='inner')

merged_df = merged_df.dropna(subset=['TYPE'])

In [16]:
cols = ['DATE', 'TYPE', 'STATE', 'CAUSE', 'TITLE', 'CATEGORY', 'DESCRIPTION', 
        'CASKLDRR', 'CASINJRR', 'CARSDMG', 'CARSHZD', 'TEMP', 'VISIBLTY', 'WEATHER', 
        'TRNSPD', 'TONS', 'TYPEQ', 'TYPTRK', 'ACCDMG', 'NARR1', 'NARR2', 'NARR3', 
        'NARR4', 'NARR5', 'NARR6', 'NARR7', 'NARR8', 'NARR9', 'NARR10', 'NARR11', 
        'NARR12', 'NARR13', 'NARR14', 'NARR15', 'Latitude', 'Longitud']
merged_df = merged_df.reindex(columns=cols)
merged_df.to_csv('./data/train_final.csv', index=False)

## Airplanes

In [8]:
ap = pd.read_csv('./data/airplanes.csv')

In [9]:
## change datetime format to match train data
ap['Date'] = pd.to_datetime(ap['Date'], format='%m/%d/%Y')
ap['Date'] = ap['Date'].dt.strftime('%Y-%m-%d')

ap.drop(columns=['Time', 'Flight #', 'Registration', 'Route', 'cn/In'], inplace=True) # drop unneeded cols

In [10]:
# ref: https://stackoverflow.com/questions/1396084/regex-for-comma-delimited-list
# https://www.rexegg.com/regex-quickstart.html
states = (
    "Alabama|Alaska|Arizona|Arkansas|California|Colorado|Connecticut|Delaware|Florida|"
    "Georgia|Hawaii|Idaho|Illinois|Indiana|Iowa|Kansas|Kentucky|Louisiana|Maine|Maryland|"
    "Massachusetts|Michigan|Minnesota|Mississippi|Missouri|Montana|Nebraska|Nevada|"
    "New Hampshire|New Jersey|New Mexico|New York|North Carolina|North Dakota|Ohio|Oklahoma|"
    "Oregon|Pennsylvania|Rhode Island|South Carolina|South Dakota|Tennessee|Texas|Utah|"
    "Vermont|Virginia|Washington|West Virginia|Wisconsin|Wyoming"
)

pattern = r",\s*(" + states + r")$" # U.S. incidents are in format city, state

state_names = ap['Location'].str.extract(pattern, expand=False) # extract state names from all rows; rows that don't contain a state are NaN
mask = state_names.notna() # remove rows that don't contain a state
ap = ap[mask]

In [11]:
def state_abbreviation(state):
    state_abbr = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
        'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
        'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
        'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
        'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
        'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
        'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
        'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
        'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    }
    return state_abbr.get(state)


ap[['City', 'State']] = ap['Location'].str.extract(r'(.*),\s*(' + states + r')$', expand=True) # separate city and state into separate cols
ap['State'] = ap['State'].apply(state_abbreviation) # change state col into abbreviated encoding

In [ ]:
cols = ['Date', 'Location', 'City', 'State', 'Operator', 'Aboard', 'Fatalities', 'Ground', 'Type','Summary']
ap.to_csv('./data/airplanes_final.csv', index=False)